In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_d4.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_b5.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ea.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_a3.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_9a.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_98.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ee.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ca.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_88.jpg
/kaggle/input/ident

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import os

In [16]:
data_dir = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset'


In [17]:
image_size = (224, 224)
batch_size = 32

In [18]:
data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Splitting the data into 80% training and 20% validation


In [19]:
dataset = data_generator.flow_from_directory(data_dir,
                                             target_size=image_size,
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             subset='training')

Found 711 images belonging to 8 classes.


In [20]:
validation_dataset = data_generator.flow_from_directory(data_dir,
                                                        target_size=image_size,
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        subset='validation')


Found 174 images belonging to 8 classes.


In [21]:
base_model = EfficientNetB4(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

In [22]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(dataset.num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization

In [23]:
model = Model(inputs=base_model.input, outputs=predictions)

In [24]:
for layer in base_model.layers:
    layer.trainable = False

In [25]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [26]:
num_train_samples = dataset.samples
num_val_samples = validation_dataset.samples
epochs = 25

history = model.fit(dataset,
                    steps_per_epoch=num_train_samples // batch_size,
                    validation_data=validation_dataset,
                    validation_steps=num_val_samples // batch_size,
                    epochs=epochs)

Epoch 1/25


2023-09-27 11:14:58.268726: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


22/22 [==============================] - 68s 2s/step - loss: 1.7126 - accuracy: 0.4271 - val_loss: 1.3008 - val_accuracy: 0.5813
Epoch 2/25
22/22 [==============================] - 42s 2s/step - loss: 1.1065 - accuracy: 0.6436 - val_loss: 1.0397 - val_accuracy: 0.6500
Epoch 3/25
22/22 [==============================] - 42s 2s/step - loss: 0.9032 - accuracy: 0.7172 - val_loss: 0.9813 - val_accuracy: 0.6875
Epoch 4/25
22/22 [==============================] - 41s 2s/step - loss: 0.8137 - accuracy: 0.7614 - val_loss: 0.9965 - val_accuracy: 0.6562
Epoch 5/25
22/22 [==============================] - 39s 2s/step - loss: 0.7514 - accuracy: 0.7555 - val_loss: 0.8957 - val_accuracy: 0.6875
Epoch 6/25
22/22 [==============================] - 41s 2s/step - loss: 0.6457 - accuracy: 0.8174 - val_loss: 0.8640 - val_accuracy: 0.7063
Epoch 7/25
22/22 [==============================] - 41s 2s/step - loss: 0.6485 - accuracy: 0.8210 - val_loss: 0.8032 - val_accuracy: 0.7437
Epoch 8/25
22/22 [=============

In [27]:
test_loss, test_accuracy = model.evaluate(validation_dataset, steps=num_val_samples // batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


5/5 [==============================] - 7s 1s/step - loss: 0.7009 - accuracy: 0.7625
Test loss: 0.7008594274520874
Test accuracy: 0.762499988079071


In [28]:
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_accuracy'][-1]


In [29]:
print('Training loss:', train_loss)
print('Training accuracy:', train_accuracy)
print('Validation loss:', val_loss)
print('Validation accuracy:', val_accuracy)

Training loss: 0.3835277855396271
Training accuracy: 0.892488956451416
Validation loss: 0.6631024479866028
Validation accuracy: 0.78125
